In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0


160.49625539779663

In [3]:
casos = casos_confirmados.get_casos()
casos.shape

(329782, 18)

In [4]:
obitos = casos_confirmados.get_obitos()
obitos.shape

(6764, 11)

In [5]:
# duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
duplicados = casos.loc[casos['hash'].isin(casos['hash_less'])].index.tolist()
print(len(duplicados))
# duplicados_more = casos.loc[casos['hash'].isin(casos['hash_more'])].index.tolist()
# print(len(duplicados_more))
# duplicados = set(duplicados_less + duplicados_more)
# print(len(duplicados))
# casos_dropar = casos.loc[dropar]
# casos_obitos_nao_dropar = casos_dropar.loc[casos_dropar['hash'].isin(obitos['hash'])].index.tolist()
# dropar = dropar - set(casos_obitos_nao_dropar)

casos = casos.rename(columns={'motivo_exclusao': 'excluir'})

casos.loc[casos['excluir'].notnull(), 'excluir'] = 'SIM'
casos.loc[casos['excluir'].isnull(), 'excluir'] = 'NAO'
casos.loc[casos['obito'].notnull(), 'obito'] = 'SIM'
casos.loc[casos['obito'].isnull(), 'obito'] = 'NAO'

casos.loc[duplicados,'excluir'] = 'SIM'
casos.loc[duplicados,'hash'] = None
casos.loc[duplicados,'hash_less'] = None
casos.loc[duplicados,'hash_more'] = None

1935


In [6]:
# obitos = obitos.rename(columns={'data_do_obito': 'data_cura_obito'})
# obitos['evolucao'] = 'OBITO'
# obitos = obitos[['evolucao','data_cura_obito','hash']]

# casos.loc[casos['rs'].isnull(), 'mun_resid'] = casos.loc[casos['rs'].isnull(), 'mun_resid'] + '/' + casos.loc[casos['rs'].isnull(), 'uf_resid']

# casos = pd.merge(left=casos, right=obitos, on='hash', how='left')

# casos['rs'] = None
# casos['is'] = None

casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir']]
casos.dtypes

ordem                   int32
ibge7                  object
nome                   object
sexo                   object
idade                   int64
mun_resid              object
mun_atend              object
rs                     object
laboratorio            object
dt_diag        datetime64[ns]
comunicacao    datetime64[ns]
is                     object
obito                  object
data_obito     datetime64[ns]
excluir                object
dtype: object

In [7]:
writer = pd.ExcelWriter("novos_casos_confirmados.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos.to_excel(writer,index=None)

writer.save()
writer.close()